In [2]:
import numpy as np
import operator
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 1. 데이터 파일 로드

def file2matrix(filename):
  file = open(filename)
  n = len(file.readlines())
  return_matrix = np.zeros((n, 3))
  class_labels = []
  file = open(filename)
  index = 0
  for line in file.readlines():
    line = line.strip()
    tokens = line.split('\t')
    return_matrix[index, :] = tokens[0:3]
    class_labels.append(tokens[-1])
    index += 1
  
  return return_matrix, class_labels

In [5]:
X, y = file2matrix('drive/MyDrive/datingTestSet.txt')

print(y)
print(X)


['largeDoses', 'smallDoses', 'didntLike', 'didntLike', 'didntLike', 'didntLike', 'largeDoses', 'largeDoses', 'didntLike', 'largeDoses', 'didntLike', 'didntLike', 'smallDoses', 'didntLike', 'didntLike', 'didntLike', 'didntLike', 'didntLike', 'smallDoses', 'largeDoses', 'smallDoses', 'didntLike', 'smallDoses', 'largeDoses', 'smallDoses', 'largeDoses', 'smallDoses', 'largeDoses', 'smallDoses', 'didntLike', 'largeDoses', 'didntLike', 'largeDoses', 'didntLike', 'smallDoses', 'didntLike', 'didntLike', 'smallDoses', 'largeDoses', 'largeDoses', 'didntLike', 'smallDoses', 'largeDoses', 'largeDoses', 'largeDoses', 'didntLike', 'didntLike', 'didntLike', 'didntLike', 'smallDoses', 'smallDoses', 'didntLike', 'largeDoses', 'smallDoses', 'smallDoses', 'smallDoses', 'smallDoses', 'largeDoses', 'didntLike', 'smallDoses', 'didntLike', 'smallDoses', 'smallDoses', 'smallDoses', 'smallDoses', 'smallDoses', 'largeDoses', 'smallDoses', 'largeDoses', 'didntLike', 'smallDoses', 'largeDoses', 'smallDoses', 'sma

In [10]:
# 2. 데이터 전처리

def apply_normalizer(dataset, offset, divisor):
  dataset_normalized = np.zeros(dataset.shape)
  N = dataset.shape[0]
  dataset_normalized = dataset - np.tile(offset, (N,1))
  dataset_normalized = dataset_normalized / np.tile(divisor, (N,1))
  
  return dataset_normalized


def normalize_minmax(dataset):
  minval = dataset.min(0)
  maxval = dataset.max(0)

  dataset_normalized = apply_normalizer(dataset, minval, maxval-minval)

  return dataset_normalized, minval, maxval-minval


def normalize_meanstd(dataset):
  meanval = dataset.mean(0)
  stdval = dataset.std(0)

  dataset_normalized = apply_normalizer(dataset, meanval, stdval)

  return dataset_normalized, meanval, stdval

In [12]:
# 3. kNN classifier 구현

def knn(x_test, X_train, labels, k):
  N_train = X_train.shape[0]

  # 1) X_train의 모든 데이터 포인트에 대해 x_test로부터의 거리 측정
  diff_mat = np.tile(x_test, (N_train, 1)) - X_train
  sq_diff_mat = diff_mat ** 2
  sq_distances = sq_diff_mat.sum(axis=1)
  distances = sq_distances ** 0.5

  # 2) x_test로부터 가장 가까운 k개의 X_train 데이터 포인트를 찾아 "이웃점 집합" 구성
  sorted_dist_indicies = distances.argsort()

  # 3) 가장 주류인 lable 따라 x_test 예측
  class_count = {}
  for i in range(k):
    vote = labels[sorted_dist_indicies[i]]
    class_count[vote] = class_count.get(vote, 0) + 1
  sorted_class_count = sorted(class_count.items(), key = operator.itemgetter(1), reverse = True)

  # 4) result 리턴
  result = sorted_class_count[0][0]
  return result

In [14]:
# 4. 테스트 데이터 떼어뒀다 예측 정확도 테스트

# 1) 데이터 재로드
X, y = file2matrix('drive/MyDrive/datingTestSet.txt')

holdout_ratio = .2

N = X.shape[0] # instance 수
N_ts = int(N*holdout_ratio) # test instance 수
N_tr = N - N_ts # training instance 수

# 2) split dataset
X_tr = X[0:N_tr,:]
y_tr = y[0:N_tr]

X_ts = X[N_tr:, :]
y_ts = y[N_tr:]

# 3) Normalize the dataset
X_normalized_tr, off, div = normalize_minmax(X_tr)
X_normalized_ts = apply_normalizer(X_ts, off, div)

# 4) Apply the classifier
n_errors = 0
y_pred_ts = []
for i in range(N_ts):
  y_pred_ts.append(knn(X_normalized_ts[i], X_normalized_tr, y_tr, 3))
  if(y_pred_ts[i] != y_ts[i]):
    n_errors += 1

print("the accurancy is : %f" % (1-n_errors/float(N_ts)))
print("the error rate is : %f" % (n_errors/float(N_ts)))

print("\n---- (Y_true, Y_pred) pairs ----")
print(*list(zip(y_ts, y_pred_ts)), sep="\n")



the accurancy is : 0.950000
the error rate is : 0.050000

---- (Y_true, Y_pred) pairs ----
('didntLike', 'didntLike')
('smallDoses', 'smallDoses')
('largeDoses', 'largeDoses')
('smallDoses', 'smallDoses')
('smallDoses', 'smallDoses')
('largeDoses', 'largeDoses')
('didntLike', 'didntLike')
('didntLike', 'didntLike')
('didntLike', 'didntLike')
('didntLike', 'smallDoses')
('largeDoses', 'largeDoses')
('smallDoses', 'smallDoses')
('smallDoses', 'smallDoses')
('didntLike', 'didntLike')
('largeDoses', 'largeDoses')
('didntLike', 'didntLike')
('smallDoses', 'smallDoses')
('largeDoses', 'largeDoses')
('didntLike', 'didntLike')
('largeDoses', 'largeDoses')
('didntLike', 'didntLike')
('largeDoses', 'largeDoses')
('didntLike', 'didntLike')
('didntLike', 'didntLike')
('largeDoses', 'largeDoses')
('smallDoses', 'smallDoses')
('largeDoses', 'largeDoses')
('didntLike', 'didntLike')
('didntLike', 'didntLike')
('largeDoses', 'largeDoses')
('largeDoses', 'largeDoses')
('largeDoses', 'largeDoses')
('larg

In [16]:
# 5. 사용자 입력 따른 매칭 점수 예상해주는 프로그램

def classifyPerson():
  resultList = ['not at all', 'in small doses', 'in large doses']

  percentTats = float(input("Percentage of time spent playing video games? "))
  ffMiles = float(input("frequent flier miles earned per year? "))
  iceCream = float(input("Liters of ice cream consumed per year? "))

  X_hist, y_hist = file2matrix('drive/MyDrive/datingTestSet.txt')

  X_hist_normalized, off, div = normalize_minmax(X_hist)
  X_test = np.array([ffMiles, percentTats, iceCream])
  classifierResult = knn((X_test - off) / div, X_hist_normalized, y_hist, 3)
  print("You will probably like this person: ", classifierResult)

classifyPerson()

Percentage of time spent playing video games? 3
frequent flier miles earned per year? 20
Liters of ice cream consumed per year? 1
You will probably like this person:  smallDoses
